In [29]:
print("Hello")

Hello


In [30]:
#import libraries
import pandas as pd
import sqlite3

In [31]:
#read csv file
cdata = pd.read_csv("orders.csv")
#display
cdata.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [32]:
#read json file
jdata = pd.read_json("users.json")
#display
jdata.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [33]:
#create temporary database in memory
conn = sqlite3.connect(':memory:')

#load and execute your sql file
with open("restaurants.sql", 'r') as f:
    conn.executescript(f.read())

#get the table name
sql_data = pd.read_sql("SELECT * FROM restaurants", conn)
conn.close()

In [93]:
#join sql with csv
merged_data = pd.merge(sql_data, cdata, on='restaurant_id', how='left')
#join merged_data with json
final_dataset = pd.merge(merged_data, jdata, on='user_id', how='left')

#view the result
print(final_dataset)
print(final_dataset.head())

      restaurant_id restaurant_name_x  cuisine  rating  order_id  user_id  \
0                 1      Restaurant_1  Chinese     4.8        93     2874   
1                 1      Restaurant_1  Chinese     4.8      1939      287   
2                 1      Restaurant_1  Chinese     4.8      2253     1521   
3                 1      Restaurant_1  Chinese     4.8      2546     2018   
4                 1      Restaurant_1  Chinese     4.8      3474     2263   
...             ...               ...      ...     ...       ...      ...   
9995            500    Restaurant_500  Chinese     3.2      8410     1275   
9996            500    Restaurant_500  Chinese     3.2      9015      452   
9997            500    Restaurant_500  Chinese     3.2      9184      464   
9998            500    Restaurant_500  Chinese     3.2      9563     1578   
9999            500    Restaurant_500  Chinese     3.2      9939     2774   

      order_date  total_amount                restaurant_name_y       name 

In [38]:
#1. which city has highest total revenue(total amount) from gold members
#filer for gold members
gold_members = final_dataset[final_dataset['membership']=='Gold']
#grp city and sum revenue
city_revenue = gold_members.groupby('city')['total_amount'].sum()
#sort values
top_city = city_revenue.idxmax()
highest_revenue = city_revenue.max()
#display
print(f"The city that has the highest total revenue (total_amount) from Gold members is {top_city} with ${highest_revenue:,.2f}")
print(f"TOP CITY : {top_city}")
print(f"Highest Revenue : ${highest_revenue}")

The city that has the highest total revenue (total_amount) from Gold members is Chennai with $1,080,909.79
TOP CITY : Chennai
Highest Revenue : $1080909.79


In [39]:
#2. cuisine which has highest average order value across all orders
#group cuisine and calc mean of total order amt
cuisine_aov = final_dataset.groupby('cuisine')['total_amount'].mean()
#get highest AOV cuisine and it's total amt
top_cuisine = cuisine_aov.idxmax()
top_value = cuisine_aov.max()
#display
print(f"TOP CUISINE : {top_cuisine}")
print(f"Highest Average Order Value : {top_value}")

TOP CUISINE : Mexican
Highest Average Order Value : 808.0213444401395


In [74]:
#3. how many distinct users placed orders worth more than $1000 in total(sum of all their orders)?
#group user_id and sum their total order amt
user_total_spending = final_dataset.groupby('user_id')['total_amount'].sum()
#filter for spending > 1000
top_spenders = user_total_spending[user_total_spending > 1000]
#count users
distinct_users_count = top_spenders.count()
print(f"TOP SPENDER's COUNT : {distinct_users_count}")


TOP SPENDER's COUNT : 2544


In [ ]:
#4. which restaurant rating range generated the highest revenue?
#1. define rating creating ranges to reduce noise
bins = [0, 3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['0-3.0', '3.0-3.5', '3.6-4.0', '4.1-4.5', '4.6-5.0',]
#create new column based on ranges
final_dataset['rating_range'] = pd.cut(final_dataset['rating'], bins=bins, labels=labels)
#sum revenue for each range
range_revenue = final_dataset.groupby('rating_range')['total_amount'].sum()
#find winning range
top_range_label = range_revenue.idxmax()
total_val = range_revenue.max()

print(f"Rating Range of Restaurants having Highest Revenue : {top_range_label}")

Rating Range of Restaurants having Highest Revenue : 4.6-5.0


C:\Users\Apurva gosavi\AppData\Local\Temp\ipykernel_17600\328890238.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  range_revenue = final_dataset.groupby('rating_range')['total_amount'].sum()


In [75]:
#5. among gold members, which city has highest avg order value
gold_city_aov = gold_members.groupby('city')['total_amount'].mean()
top_gold_city = gold_city_aov.idxmax()
top_gold_value = gold_city_aov.max()
print(f"Highest AOV having city among GOLD members : {top_gold_city}")


Highest AOV having city among GOLD members : Chennai


In [80]:
#6. Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?
cuisine_counts = final_dataset.groupby('cuisine')['restaurant_name_y'].nunique()
cuisine_revenue = final_dataset.groupby('cuisine')['total_amount'].sum()
cuisine_summary = pd.DataFrame({
    'distinct_restaurants' : cuisine_counts,
    'total_revenue' : cuisine_revenue
})
result = cuisine_summary.sort_values(by=['distinct_restaurants', 'total_revenue'], ascending=[True, False])
print(f"Cuisine which has the lowest number of distinct restaurants but still contributes significant revenue : ")
print(result.head())

Cuisine which has the lowest number of distinct restaurants but still contributes significant revenue : 
         distinct_restaurants  total_revenue
cuisine                                     
Chinese                   115     1930504.65
Indian                    122     1971412.58
Mexican                   124     2085503.09
Italian                   124     2024203.80


In [76]:
#7. What percentage of total orders were placed by Gold members? (Rounded to nearest integer)
#count of orders by gold members
gold_order_count = len(final_dataset[final_dataset['membership']=='Gold'])
#total number od orders(count)
total_order_count = len(final_dataset)
#calc percentage 
percent_gold = (gold_order_count / total_order_count) * 100
#round to nearest integer
final_percent = int(round(percent_gold))
print(f"Rounded Percentage of Total Orders placed by Gold Members : {final_percent}%")


Rounded Percentage of Total Orders placed by Gold Members : 50%


In [91]:
keyword = "Ruchi Mess"
search = final_dataset[final_dataset['restaurant_name_y'].str.contains(keyword, case=False, na=False)]
print(search[['restaurant_name_y', 'total_amount']])

            restaurant_name_y  total_amount
2272        Ruchi Mess Andhra       1478.62
2273        Ruchi Mess Andhra        926.46
2274        Ruchi Mess Andhra       1312.52
2275        Ruchi Mess Andhra        988.35
2276        Ruchi Mess Andhra        958.56
...                       ...           ...
7766  Ruchi Mess Multicuisine        305.04
7767  Ruchi Mess Multicuisine        400.76
7768  Ruchi Mess Multicuisine        944.34
7769  Ruchi Mess Multicuisine       1403.62
7770  Ruchi Mess Multicuisine       1186.33

[115 rows x 2 columns]


In [ ]:
#8. Which restaurant has the highest average order value but less than 20 total orders?

option = [
    'Grand Cafe Punjabi',
    'Grand Restaurant South Indian',
    'Ruchi Mess Multicuisine',
    'Ruchi Foods Chinese'
]
filtered_data = final_dataset[final_dataset['restaurant_name_y'].isin(option)]
avg_values = filtered_data.groupby('restaurant_name_y')['total_amount'].mean()
order_counts = filtered_data.groupby('restaurant_name_y')['total_amount'].count()
summary = pd.DataFrame({
    'avg_order_val' : avg_values,
    'total_orders' : order_counts
})
filtered_summary = summary[summary['total_orders'] < 20]
winner = filtered_summary.sort_values(by='avg_order_val', ascending=False).head(1)
print(f"TOP RESTAURANT : {winner}")

TOP RESTAURANT :                           avg_order_val  total_orders
restaurant_name_y                                    
Hotel Dhaba Multicuisine    1040.222308            13


In [ ]:
#9. Which combination contributes the highest revenue?
#valid_memberships = ['Gold', 'Regular']
#valid_cuisines = ['Indian', 'Italian', 'Chinese']
#filtered_option_data = final_dataset[(final_dataset['membership'].isin(valid_memberships)) & (final_dataset['cuisine'].isin(valid_cuisines))]
combo_revenue = final_dataset.groupby(['membership', 'cuisine'])['total_amount'].sum()
best_combo = combo_revenue.idxmax()
highest_value = combo_revenue.max()
print(f"Combination : {best_combo[0]} & {best_combo[1]}")

Combination : Regular & Mexican


In [ ]:
#9. Which combination contributes the highest revenue?
options = [
    ('Gold', 'Indian'),
    ('Gold', 'Italian'),
    ('Regular', 'Indian'),
    ('Regular', 'Chinese')
]
results = {}
for member, food in options:
    revenue = final_dataset[(final_dataset['membership']==member) & (final_dataset['cuisine']==food)]['total_amount'].sum()
    results[(member, food)] = revenue

winner = max(results, key=results.get)
print(f"Combination : {winner}")

Combination : ('Gold', 'Italian')


In [ ]:
#10. During which quarter of the year is the total revenue highest?
final_dataset['order_date'] = pd.to_datetime(final_dataset['order_date'])
final_dataset['quarter'] = final_dataset['order_date'].dt.quarter
quarterly_revenue = final_dataset.groupby('quarter')['total_amount'].sum()
best_quarterly_num = quarterly_revenue.idxmax()
highest_revenue = quarterly_revenue.max()
quarterly_labels = {1: "Jan-Mar", 2: "Apr-Jun", 3: "Jul-Sep", 4: "Oct-Dec"}
print(f"Quarter with Highest Revenue : {quarterly_labels[best_quarterly_num]}") 

C:\Users\Apurva gosavi\AppData\Local\Temp\ipykernel_17600\3299363590.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_dataset['order_date'] = pd.to_datetime(final_dataset['order_date'])


Quarter with Highest Revenue : Jul-Sep


In [77]:
#11. How many total orders were placed by users with Gold membership?
print(gold_order_count)

4987


In [ ]:
#12. What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?
final_dataset['city'] = final_dataset['city'].astype(str).str.strip().str.title()
hyd_data = final_dataset[final_dataset['city']=='Hyderabad']
total_order_hyd = len(hyd_data)
total_revenue_hyd = hyd_data['total_amount'].sum()
round_revenue_hyd = int(round(total_revenue_hyd))
print(f"Total Revenue from Hyderabad : {round_revenue_hyd}")

Total Revenue from Hyderabad : 1889367


In [78]:
#13. How many distinct users placed at least one order?
distinct_users = final_dataset['user_id'].nunique()
print(f'Distinct User Count : {distinct_users}')

Distinct User Count : 2883


In [ ]:
#14. What is the average order value (rounded to 2 decimals) for Gold members?
final_dataset['total_amount'] = pd.to_numeric(final_dataset['total_amount'], errors='coerce')
avg_aov_gold = gold_members['total_amount'].mean()
print(f'Average Order Value for Gold Members : {avg_aov_gold:.2f}')

Average Order Value for Gold Members : 797.15


In [ ]:
#15. How many orders were placed for restaurants with rating ≥ 4.5?
final_dataset['rating'] = pd.to_numeric(final_dataset['rating'], errors='coerce')
high_rated_orders = final_dataset[final_dataset['rating'] >= 4.5]
total_high_rated_orders = len(high_rated_orders)
print(f"Number of Orders placed at restaurants rated >= 4.5 : {total_high_rated_orders}")

Number of Orders placed at restaurants rated >= 4.5 : 3374


In [ ]:
#16. How many orders were placed in the top revenue city among Gold members only?
top_city_name = gold_members.groupby('city')['total_amount'].sum().idxmax()
top_city_orders = len(gold_members[gold_members['city']== top_city_name])
print(f"Total Orders for this city : {top_city_orders}")

Total Orders for this city : 1337
